<a href="https://colab.research.google.com/github/siting1206/NLP_HW2/blob/main/NLP_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
train_df = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/train_ret.txt', sep='\t')
valid_df = pd.read_csv('/content/drive/MyDrive/NLP_assignment2/Dataset/valid_ret.txt', sep='\t')
train_df[['snippet','question','answer']] = train_df[['snippet','question','answer']].apply(lambda x: x.str.strip('\"'))
valid_df[['snippet','question','answer']] = valid_df[['snippet','question','answer']].apply(lambda x: x.str.strip('\"'))

In [53]:
train_df.head()

,snippet,question,answer
0,"lebowski sweater replica jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus
1,"assignment 13 nov 30 , 2014 2 1912 olympian fo...",2 1912 olympian football star carlisle indian ...,jim thorpe
2,"weather world records average , yuma receives ...","city yuma state record average 4 , 055 hours s...",arizona
3,gravestone famous ancestor roger sherman signe...,"signer dec indep , framer constitution mass , ...",john adams
4,"assignment 13 nov 30 , 2014 united states 6 ti...","title aesop fable , insect shared billing gras...",ant


In [54]:
valid_df.head()

,snippet,question,answer
0,free flashcards authors studystack spain 1959 ...,"spain 1959 , wrote dangerous summer , story ri...",hemingway
1,lowest highest death valley mt whitney walk ba...,valley 282 feet sea level state lowest point w...,california
2,"banking peril barron 's apr 5 , 2010 take hub ...","like banks , many grocery stores dispensing ca...",atms
3,direct essays mickey mouse steamboat willie ka...,voice mickey mouse steamboat willie,walt disney
4,"old world capitals eastern europe vacations , ...",eastern european capital city 2 2 million,bucharest


In [55]:
# train_df['ans_start'] = [y.index(x) if str(x) in y else 0 for x,y in zip(train_df["answer"], train_df["snippet"])]
# train_df['ans_end'] = [x+len(y)-1 if x!=0 else 0 for x,y in zip(train_df["ans_start"], train_df["answer"])]

In [56]:
train_df['ans_start'] = [y.index(x)+len(str(z)) if str(x) in y else 0 for x,y,z in zip(train_df["answer"], train_df["snippet"], train_df["question"])]
train_df['ans_end'] = [x+len(y)-1 if x!=0 else 0 for x,y,z in zip(train_df["ans_start"], train_df["answer"], train_df["question"])]

In [57]:
valid_df['ans_start'] = [y.index(x)+len(str(z)) if str(x) in y else 0 for x,y,z in zip(valid_df["answer"], valid_df["snippet"], valid_df["question"])]
valid_df['ans_end'] = [x+len(y)-1 if x!=0 else 0 for x,y,z in zip(valid_df["ans_start"], valid_df["answer"], valid_df["question"])]

In [58]:
train_df.head()

,snippet,question,answer,ans_start,ans_end
0,"lebowski sweater replica jun 17 , 2013 history...","last 8 years life , galileo house arrest espou...",copernicus,177,186
1,"assignment 13 nov 30 , 2014 2 1912 olympian fo...",2 1912 olympian football star carlisle indian ...,jim thorpe,0,0
2,"weather world records average , yuma receives ...","city yuma state record average 4 , 055 hours s...",arizona,0,0
3,gravestone famous ancestor roger sherman signe...,"signer dec indep , framer constitution mass , ...",john adams,252,261
4,"assignment 13 nov 30 , 2014 united states 6 ti...","title aesop fable , insect shared billing gras...",ant,0,0


In [59]:
valid_df.head()

,snippet,question,answer,ans_start,ans_end
0,free flashcards authors studystack spain 1959 ...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,163,171
1,lowest highest death valley mt whitney walk ba...,valley 282 feet sea level state lowest point w...,california,0,0
2,"banking peril barron 's apr 5 , 2010 take hub ...","like banks , many grocery stores dispensing ca...",atms,0,0
3,direct essays mickey mouse steamboat willie ka...,voice mickey mouse steamboat willie,walt disney,0,0
4,"old world capitals eastern europe vacations , ...",eastern european capital city 2 2 million,bucharest,0,0


## Tokenizer

In [60]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [61]:
train_data_s = train_df['snippet'].tolist()
valid_data_s = valid_df['snippet'].tolist()

train_data_q = train_df['question'].tolist()
valid_data_q = valid_df['question'].tolist()
# for i in range(0,len(train_data_q)):
#   if type(train_data_q[i]) == float:
#     print(i)

In [62]:
train_encodings = tokenizer(train_data_q, train_data_s, truncation=True, padding=True)
val_encodings = tokenizer(valid_data_q, valid_data_s, truncation=True, padding=True)

In [63]:
tokenizer.decode(train_encodings['input_ids'][0])

"[CLS] last 8 years life, galileo house arrest espousing man's theory [SEP] lebowski sweater replica jun 17, 2013 history last 8 years life, galileo house arrest espousing man's theory copernicus espn's top 10 time [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [64]:
tokenizer.decode(val_encodings['input_ids'][0])

'[CLS] spain 1959, wrote dangerous summer, story rival bullfighters [SEP] free flashcards authors studystack spain 1959, wrote dangerous summer, story rival bullfighters, hemingway 1884 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [65]:
train_answer = train_df[['ans_start', 'ans_end']].to_dict('records')
valid_answer = valid_df[['ans_start', 'ans_end']].to_dict('records')

In [66]:
def add_token_positions(encodings, answers):
    ans_start, ans_end = [],[]

    for i in range(len(answers)):
        ans_start.append(encodings.char_to_token(i, answers[i]['ans_start'], 0))
        ans_end.append(encodings.char_to_token(i, answers[i]['ans_end'], 0))

        if ans_start[-1] is None:
            ans_start[-1] = 0
            ans_end[-1] = 0
            # continue

        shift = 1
        while ans_end[-1] is None:
            ans_end[-1] = encodings.char_to_token(i, answers[i]['ans_end'] - shift)
            shift += 1
    encodings.update({'ans_start':ans_start, 'ans_end':ans_end})

In [67]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)
add_token_positions(val_encodings, valid_answer)

In [68]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'ans_start', 'ans_end'])

In [69]:
print(train_encodings['ans_start'][0])
print(train_encodings['ans_end'][0])

0
0


In [70]:
class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [71]:
train_dataset = qrDataset(train_encodings)
valid_dataset = qrDataset(val_encodings)

In [72]:
next(iter(train_dataset))

{'input_ids': tensor([  101,  1314,   129,  1201,  1297,   117, 20003,  4759,  1186,  1402,
          6040, 13936,  5674, 19578,  1299,   112,   188,  2749,   102,  5837,
         14251,  5437, 15195, 16498,   179,  3488,  1542,   117,  1381,  1607,
          1314,   129,  1201,  1297,   117, 20003,  4759,  1186,  1402,  6040,
         13936,  5674, 19578,  1299,   112,   188,  2749, 16743,  4558, 14191,
         13936,  1643,  1179,   112,   188,  1499,  1275,  1159,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

## Model

In [73]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.fc = nn.Linear(768, 2)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out

## Train

In [74]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-4)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a f

In [75]:
# Pack data into dataloader by batch
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [76]:
training_epoch = 2

In [77]:
loss_fct = CrossEntropyLoss()

In [78]:
def evaluate(valid_loader):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        loop = tqdm(valid_loader, leave=True)
        for batch_id, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            ans_start = batch['ans_start'].to(device)
            ans_end = batch['ans_end'].to(device)

            # model output
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

            ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
            ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

            ans_start_loss = loss_fct(ans_start_logits, ans_start)
            ans_end_loss = loss_fct(ans_end_logits, ans_end)

            loss = ans_start_loss + ans_end_loss

            running_loss += loss.item()
            if batch_id % 30 == 0 and batch_id != 0:
                print('Validation Epoch {} Batch {} Loss {:.4f}'.format(
                    batch_id + 1, batch_id, running_loss / 30))
                running_loss = 0.0

        print("evaluate loss: ", loss)

In [79]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        ans_start = batch['ans_start'].to(device)
        ans_end = batch['ans_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

        ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
        ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

        ans_start_loss = loss_fct(ans_start_logits, ans_start)
        ans_end_loss = loss_fct(ans_end_logits, ans_end)



        loss = ans_start_loss + ans_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 100 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 100))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    evaluate(valid_loader)

Epoch 0:   1%|          | 101/12477 [00:26<55:58,  3.68it/s, loss=1.7]

Epoch 101 Batch 100 Loss 1.9316


Epoch 0:   2%|▏         | 201/12477 [00:54<55:43,  3.67it/s, loss=1.35]

Epoch 201 Batch 200 Loss 1.4167


Epoch 0:   2%|▏         | 301/12477 [01:21<54:41,  3.71it/s, loss=1.34]

...

Validation Epoch 1681 Batch 1680 Loss 10.1997


 99%|█████████▊| 1712/1737 [02:16<00:01, 12.61it/s]

Validation Epoch 1711 Batch 1710 Loss 10.1997


100%|██████████| 1737/1737 [02:18<00:00, 12.58it/s]

evaluate loss:  tensor(10.1997, device='cuda:0')


In [80]:
torch.save(model.state_dict(), '/content/drive/MyDrive/NLP_assignment2/aicup_model')

In [81]:
model = myModel().to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP_assignment2/aicup_model'))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

## Predict

In [82]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    ans_sub_output = []

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        ans_start_logits, ans_end_logits = torch.split(outputs, 1, 2)

        ans_start_logits = ans_start_logits.squeeze(-1).contiguous()
        ans_end_logits = ans_end_logits.squeeze(-1).contiguous()

        ans_start_prdict = torch.argmax(ans_start_logits, 1).cpu().numpy()
        ans_end_prdict = torch.argmax(ans_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((ans_start_prdict[i].item(), ans_end_prdict[i].item()))

            ans_sub = tokenizer.decode(input_ids[i][ans_start_prdict[i]:ans_end_prdict[i]+1])
            
            ans_sub_output.append(ans_sub)
    
    return ans_sub_output, predict_pos

In [83]:
ans_sub_output, predict_pos = predict(valid_loader)

100%|██████████| 1737/1737 [02:19<00:00, 12.49it/s]


In [84]:
def get_output_post_fn(test, ans_sub_output):
    ans_sub = []
    for i in range(len(test)):

        ans_sub_pred = ans_sub_output[i].split()

        if ans_sub_pred is None:
            ans_sub_pred = []
        ans_sub_error_index = ans_sub_pred.index('[SEP]') if '[SEP]' in ans_sub_pred else -1

        if ans_sub_error_index != -1:
            ans_sub_pred = ans_sub_pred[:ans_sub_error_index]


        ans_sub.append(' '.join(ans_sub_pred))

    return ans_sub

In [85]:
ans_sub = get_output_post_fn(valid_df, ans_sub_output)

In [86]:
valid_df['ans_sub'] = ans_sub

In [87]:
valid_df.head()

,snippet,question,answer,ans_start,ans_end,ans_sub
0,free flashcards authors studystack spain 1959 ...,"spain 1959 , wrote dangerous summer , story ri...",hemingway,163,171,
1,lowest highest death valley mt whitney walk ba...,valley 282 feet sea level state lowest point w...,california,0,0,
2,"banking peril barron 's apr 5 , 2010 take hub ...","like banks , many grocery stores dispensing ca...",atms,0,0,"? secret leaf lard, rendered 8 hours"
3,direct essays mickey mouse steamboat willie ka...,voice mickey mouse steamboat willie,walt disney,0,0,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...
4,"old world capitals eastern europe vacations , ...",eastern european capital city 2 2 million,bucharest,0,0,name


In [88]:
def nltk_token_string(sentence):
    # print(sentence)
    tokens = nltk.word_tokenize(sentence)
    for i in range(len(tokens)):
        if len(tokens[i]) == 1:
            tokens[i] = re.sub(r"[!\"#$%&\'()*\+, -.\/:;<=>?@\[\\\]^_`{|}~]", '', tokens[i])
    while '' in tokens:
        tokens.remove('')
    tokens = ' '.join(tokens)
    return tokens

In [89]:
def lcs(X, Y):
    X_, Y_ = [], []
    
    X_ = nltk_token_string(X)
    Y_ = nltk_token_string(Y)

    m = len(X_)
    n = len(Y_)
 
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
 
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X_[i-1] == Y_[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
 
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]


def acc(full, sub):
    common = lcs(full, sub)
    union = len(full) + len(sub) - common
    accuracy = float(common/union)

    return accuracy

In [90]:
ans_acc_sum = 0
for i in range(valid_df.shape[0]):
    ans_accuracy = acc(valid_df.iloc[i]["answer"], valid_df.iloc[i]['ans_sub'])

    ans_acc_sum += ans_accuracy

print("answer accuracy: ", ans_acc_sum/valid_df.shape[0])

answer accuracy:  0.05271999862862745
